# Sprint アンサンブル学習

# 1.このSprintについて

## Sprintの目的  
・アンサンブル学習について理解する

## どのように学ぶか  
・スクラッチでアンサンブル学習の各種手法を実装していきます。

# 2.アンサンブル学習  
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。  
・ブレンディング  
・バギング  
・スタッキング  

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。  
House Prices: Advanced Regression Techniques  
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。  
train.csvを学習用（train）8割、検証用（val）2割に分割してください。

### scikit-learn

単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation  
sklearn.svm.SVR — scikit-learn 0.21.3 documentation  
sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

# 3.ブレンディング

## 【問題1】ブレンディングのスクラッチ実装  
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

### ブレンディングとは

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。  
最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）  
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）  
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）  

重要なのはそれぞれのモデルが大きく異なることです。  

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。  

《補足》  
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./HousePrice_train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
drop_df = df
# 半分以上欠損値を含む列を削除する。
drop_df = drop_df.dropna(thresh=df.shape[0]*0.5, axis=1)

# 欠損値があるサンプル（行）は削除する。
drop_df = drop_df.dropna(axis=0)

In [4]:
df_data = drop_df.loc[:,["GrLivArea","YearBuilt","SalePrice"]]
df_data

,GrLivArea,YearBuilt,SalePrice
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000
6,1694,2004,307000
...,...,...,...
1447,2090,1995,240000
1451,1578,2008,287090
1455,1647,1999,175000
1456,2073,1978,210000


In [5]:
import numpy as np
# ndarrayへ変換
X = np.array(df_data.iloc[:,0:2])
y = np.array(df_data.iloc[:,2])

In [6]:
# 訓練データと検証データの分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 47)

#### train_test_splitを実行するたびに、X_train, X_test, y_train, y_testが変動するためMSEスコアが変動するため保存する。

In [174]:
import pickle

with open("X_train.pkl", "wb") as f:
    pickle.dump(X_train, f)

with open("X_test.pkl", "wb") as f:
    pickle.dump(X_test, f)
    
with open("y_train.pkl", "wb") as f:
    pickle.dump(y_train, f)
    
with open("y_test.pkl", "wb") as f:
    pickle.dump(y_test, f)

In [7]:
# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train)
ss_X_train = ss.transform(X_train)
ss_X_test = ss.transform(X_test)

In [8]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression as LR # ★
from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor as RFR # ★
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVR  # ★
#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor as DTR # ★
from sklearn.metrics import mean_squared_error

In [9]:
#ここではロジスティック回帰、ランダムフォレスト分類器、ガウシアンナイーブベベイズを用いる。
clf1 = LR()
clf2 = DTR()
clf3 = SVR()
clf4 = RFR()


### 単一モデルで検証

#### ＜線形回帰＞

In [10]:
# 学習
ss_clf = clf1.fit(ss_X_train, y_train)

# 推定
y_pred_LR = clf1.predict(ss_X_test)

# 評価
err_LR = mean_squared_error(y_test, y_pred_LR)
err_LR


5073056118.281545

#### ＜決定木＞

In [11]:
# 学習
ss_clf = clf2.fit(ss_X_train, y_train)

# 推定
y_pred_DTR = clf2.predict(ss_X_test)

# 評価
err_DTR = mean_squared_error(y_test, y_pred_DTR)
err_DTR


4161174406.321918

#### ＜SVM＞

In [12]:
# 学習
ss_clf = clf3.fit(ss_X_train, y_train)

# 推定
y_pred_SVR = clf3.predict(ss_X_test)

# 評価
err_SVR = mean_squared_error(y_test, y_pred_SVR)
err_SVR


9511112479.715921

#### ＜ランダムフォレスト＞

In [13]:
# 学習
ss_clf = clf4.fit(ss_X_train, y_train)

# 推定
y_pred_RFR = clf4.predict(ss_X_test)

# 評価
err_RFR = mean_squared_error(y_test, y_pred_RFR)
err_RFR


2809452173.369651

以上の単体モデルをブレンディングする。

#### ＜0.3 × 線形回帰 + 0.4 × 決定木 + 0.3 × SVM＞

In [14]:
y_pred_blend1 = 0.3 * y_pred_LR + 0.4 * y_pred_DTR + 0.3 * y_pred_SVR
# 評価
err_blend1 = mean_squared_error(y_test, y_pred_blend1)
err_blend1

4078021280.192417

#### ＜0.4 × 線形回帰 + 0.5 × 決定木 + 0.1 × SVM＞

In [15]:
y_pred_blend2 = 0.4 * y_pred_LR + 0.5 * y_pred_DTR + 0.1 * y_pred_SVR
# 評価
err_blend2 = mean_squared_error(y_test, y_pred_blend2)
err_blend2

3584402681.8860116

#### ＜ロス比率から重み算出＞

In [16]:
err_brend = err_LR + err_DTR + err_SVR
y_pred_blend3 = (err_brend - err_LR)/(2 * err_brend) * y_pred_LR + (err_brend - err_DTR)/(2 * err_brend) * y_pred_DTR + (err_brend - err_SVR)/(2 * err_brend) * y_pred_SVR
# 評価
err_blend3 = mean_squared_error(y_test, y_pred_blend3)
err_blend3

3932659618.4345016

以上の結果から

| 単体モデル                 |                  |
|----------------------------|------------------|
| ①線形回帰                  | 5073056118.28154 |
| ②決定木                    | 4161174406.32191 |
| ③SVM                       | 9511112479.71592 |


| ブレンドモデル             |                  |
|----------------------------|------------------|
|（１）0.3 × ① + 0.4×② + 0.3 × ③ | 4078021280.19241 |
|（２）0.4 × ① + 0.5×② + 0.1 × ③ | 3584402681.88601 |
|（３）ロス比率の係数で重み付け  | 3932659618.4345 |

#### すべてのブレンドモデルにおいて、各構成でモデル単体モデルよりも精度が出ている。

# 4.バギング

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

In [17]:
# 標準化
ss = StandardScaler()
ss.fit(X)
ss_X = ss.transform(X)

In [18]:
from sklearn.linear_model import LinearRegression as LR

# 決定木
model_LR = LR()

X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

# 空行列作成
y_test_bag.reshape(-1,1)
y_pred = np.zeros((y_test_bag.shape[0], 1))

n_iter = 5

for i in range(n_iter):
    # 分割
    X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

    # 学習
    clf = model_LR.fit(X_train_bag, y_train_bag)

    # 推定
    y_pred_WK = model_LR.predict(X_test_bag).reshape(-1,1)

    # predict arrayに追加
    y_pred = np.concatenate([y_pred, y_pred_WK], axis = 1)

print("befor delete y_pred.shape = {}".format(y_pred.shape))
y_pred = np.delete(y_pred, 0, axis = 1)
print("after delete y_pred.shape = {}".format(y_pred.shape))
    
# 推定結果の平均
y_pred_DT = np.mean(y_pred, axis = 1).reshape(-1,1)
print("y_pred_DT.shape = {}".format(y_pred_DT.shape))

# 評価
err_DT = mean_squared_error(y_test, y_pred_DT)
err_DT

befor delete y_pred.shape = (146, 6)
after delete y_pred.shape = (146, 5)
y_pred_DT.shape = (146, 1)


7864439110.391713

In [19]:
from sklearn.tree import DecisionTreeClassifier

# 決定木
model_LR_bag = LR()

# 標準化
ss = StandardScaler()
ss.fit(X)
ss_X = ss.transform(X)

X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

# 空行列作成
y_test_bag.reshape(-1,1)
y_pred = np.zeros((y_test_bag.shape[0], 1))
y_true = np.zeros((y_test_bag.shape[0], 1))


n_iter = 5

for i in range(n_iter):
    # 分割
    X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

    # 学習
    model_LR_bag.fit(X_train_bag, y_train_bag)

    # 推定
    y_pred_WK = model_LR_bag.predict(X_test_bag).reshape(-1,1)
    y_test_bag = y_test_bag.reshape(-1,1)
    
    y_pred = np.concatenate([y_pred, y_pred_WK], axis = 1)
    y_true = np.concatenate([y_true, y_test_bag], axis = 1)

# 正解データ、予測データの最初の列はダミーデータが入っているので削除する。
y_pred = np.delete(y_pred, 0, axis = 1)
y_true = np.delete(y_true, 0, axis = 1)

    
# 正解データ、推定結果の平均
y_pred_LR = np.mean(y_pred, axis = 1).reshape(-1,1)
y_true = np.mean(y_true, axis = 1).reshape(-1,1)
print("y_pred_DT.shape = {}".format(y_pred_LR.shape))

# 評価
err_LR_bag = mean_squared_error(y_true, y_pred_DT)
err_LR_bag

y_pred_DT.shape = (146, 1)


2326674755.3579326

以上をまとめると

| 単一モデル                |                   |
|---------------------------|-------------------|
| モデル                    | MSE               |
| ③線形回帰 | 7864439110.391713 |


| アンサンブル(バギング)    |                   |
|---------------------------|-------------------|
| モデル                    | MSE               |
| ③線形回帰 | 2326674755.35793 |

##### 以上の結果となり、線形回帰の単体のMSEが7864439110.39171であるのに対して、  
##### バギングを行った後のMSEは、2326674755.35793となり、3.38倍以上の精度向上が見られた。

# 5.スタッキング

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### スタッキングとは

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。  
まずは 
$K_0$= 3, $M_0$= 2
 程度にします。

#### 《学習時》

（ステージ 0 ）

・学習データを $K_0$個に分割する。  
・分割した内の ($K_0$ − 1) 個をまとめて学習用データ、残り 1個を推定用データとする組み合わせが $K_0$ 個作れる。  
・あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。  
・それぞれの学習済みモデルに対して、使っていない残り 1 個の推定用データを入力し、推定値を得る。  
（これをブレンドデータと呼ぶ）  
・さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、   
$M_0$ 個のブレンドデータが得られる。  

（ステージ n ）

・ステージ n − 1 のブレンドデータを $M_n−1$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。  
以下同様である。

（ステージ N ）＊最後のステージ

・ステージ N − 1 の $M_{N−1}$ 個のブレンドデータを$M_{N−1}$ 次元の特徴量の入力として、  
1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

#### 《推定時》

（ステージ 0 ）

・テストデータを 
$K_0$ × $M_0$ 個の学習済みモデルに入力し、$K_0$ × $M_0$ 個の推定値を得る。  
これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ n ）

・ステージ n−1 で得たブレンドテストを $K_n$ × $M_n$ 個の学習済みモデルに入力し、$K_n$ × $M_n$ 個の推定値を得る。  
これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ N ）＊最後のステージ

・ステージ N − 1 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

### 《学習時》

In [69]:
# クロスバリデーション
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso

X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
model_ins = []
i = 0
y_brend1 = None
y_brend2 = None
y_brend3 = None
y_brend4 = None

model_ins1 = []
model_ins2 = []


# << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

# 分割数
K0= 3

# モデル数
M0 = 2

# クロスバリデーション
kf = KFold(n_splits = K0)
kf.get_n_splits(ss_X)

# 変数初期化
y_brend = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], M0).reshape(-1, M0)
model_ins_0 = np.empty(K0 * M0).reshape(K0, M0)
model_ins_0 = model_ins_0.astype(np.object)

# インスタンス作成
for ins in range(K0):
    
    model_ins_0[ins,0] = SVR()
    model_ins_0[ins,1] = RFR()


# データ分割
for i, (train_index, test_index) in enumerate(kf.split(ss_X)):
    print("i = ", i)
    #print("TRAIN:", train_index, "TEST:", test_index)

        
    X_train, X_test = ss_X[train_index], ss_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)
    print("X_train.shape = {}".format(X_train.shape))
    print("X_test.shape = {}".format(X_train.shape))
    print("y_train.shape = {}".format(y_train.shape))
    print("y_test.shape = {}".format(y_test.shape))
    
    
    for j in range(M0):
        # 学習
        model_ins_0[i,j].fit(X_train, y_train)

        # 推定
        y_brend[test_index, j] = model_ins_0[i,j].predict(X_test).flatten()
    
#print("★★★y_brend = \n{}".format(y_brend))
#print("★★★y_brend.shape = {}".format(y_brend.shape))


# << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
Kn = 3
Mn = 2
n_cnt = 1
kf = KFold(n_splits = Kn)
kf.get_n_splits(X)

# 変数初期化
model_ins_n = np.empty(Kn * Mn).reshape(Kn, Mn)
model_ins_n = model_ins_n.astype(np.object)


# インスタンス作成
for ins in range(K0):
    
    model_ins_n[ins,0] = LR()
    model_ins_n[ins,1] = DTR()

# n回ループ
for n in range(n_cnt):
    for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

        X_train, X_test = y_brend[train_index], y_brend[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)


        for j in range(Mn):
            # 学習
            model_ins_n[i,j].fit(X_train, y_train)

            # 推定
            y_brend[test_index, j] = model_ins_n[i,j].predict(X_test).flatten()


# << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
Kl = 3
Ml = 1
n_cnt = 1
kf = KFold(n_splits = Kl)
kf.get_n_splits(X)

# 変数初期化
y_brend_fin = np.zeros(ss_X.shape[0] * Ml).reshape(ss_X.shape[0], Ml).reshape(-1, Ml)
model_ins_l = np.empty(Kl * Ml).reshape(Kl, Ml)
model_ins_l = model_ins_l.astype(np.object)


# インスタンス作成
for ins in range(Kl):
    # model_ins_l[ins,0] = Lasso(alpha=0.1)
    model_ins_l[ins,0] = RFR()
    # model_ins_l[ins,0] = GaussianNB()
    
for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

    X_train, X_test = y_brend[train_index], y_brend[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

    # 学習
    model_ins_l[i,0].fit(X_train, y_train)

    # 推定
    y_brend_fin[test_index, 0] = model_ins_l[i,0].predict(X_test).flatten()
    
    
print("y_brend_fin.shape = ", y_brend_fin.shape)

i =  0
X_train.shape = (388, 2)
X_test.shape = (388, 2)
y_train.shape = (388,)
y_test.shape = (195,)
i =  1
X_train.shape = (389, 2)
X_test.shape = (389, 2)
y_train.shape = (389,)
y_test.shape = (194,)
i =  2
X_train.shape = (389, 2)
X_test.shape = (389, 2)
y_train.shape = (389,)
y_test.shape = (194,)
y_brend_fin.shape =  (583, 1)


### 《推定時》

In [70]:
# << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
predict_model_0 = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], -1)
for m in range(M0):
    predict_instance_0 = np.zeros(ss_X.shape[0] * K0).reshape(ss_X.shape[0], -1)
    
    for k in range(K0):
        # 推定
        predict_instance_0[:,k] = model_ins_0[k,m].predict(ss_X)
        
    predict_model_0[:,m] = np.mean(predict_instance_0, axis = 1).flatten()

# << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
# n回ループ
for n in range(n_cnt):
    predict_model_n = np.zeros(ss_X.shape[0] * Mn).reshape(ss_X.shape[0], -1)
    predict_model_n = predict_model_0
    for m in range(M0):
        predict_instance_n = np.zeros(ss_X.shape[0] * Kn).reshape(ss_X.shape[0] , -1)

        for k in range(K0):
            predict_instance_n[:,k] = model_ins_n[k,m].predict(predict_model_n)
            
        predict_model_n[:,m] = np.mean(predict_instance_n, axis = 1).flatten()
        
# << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
# 推定
y_predict_fin = model_ins_l[i,0].predict(predict_model_n).flatten()
y_predict_fin.reshape(ss_X.shape[0], -1)

print("y_predict_fin.shape", y_predict_fin.shape)


y_predict_fin.shape (583,)


In [71]:
# 評価
MSE = mean_squared_error(y, y_predict_fin)
MSE

2669079509.707969

### 以上より、スタッキングを用いるとMSE = 2669079509.707969 となり、最善の結果が出た。